# Proyecto agrupación

## Primer parte

In [1]:
!pip install requests
import requests


In [2]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
req = requests.get(url)
soup = BeautifulSoup(req.text)
data = soup.find_all("table")[0]
tabla = pd.read_html(str(data))[0]


In [5]:
tabla.head()

,0,1,2,3,4,5,6,7,8
0,M1A Not assigned,M2A Not assigned,M3A North York (Parkwoods),M4A North York (Victoria Village),M5A Downtown Toronto (Regent Park / Harbourfront),M6A North York (Lawrence Manor / Lawrence Heig...,M7A Queen's Park (Ontario Provincial Government),M8A Not assigned,M9A Etobicoke (Islington Avenue)
1,M1B Scarborough (Malvern / Rouge),M2B Not assigned,M3B North York (Don Mills) North,M4B East York (Parkview Hill / Woodbine Gardens),"M5B Downtown Toronto (Garden District, Ryerson)",M6B North York (Glencairn),M7B Not assigned,M8B Not assigned,M9B Etobicoke (West Deane Park / Princess Gard...
2,M1C Scarborough (Rouge Hill / Port Union / Hig...,M2C Not assigned,M3C North York (Don Mills) South (Flemingdon P...,M4C East York (Woodbine Heights),M5C Downtown Toronto (St. James Town),M6C York (Humewood-Cedarvale),M7C Not assigned,M8C Not assigned,M9C Etobicoke (Eringate / Bloordale Gardens / ...
3,M1E Scarborough (Guildwood / Morningside / Wes...,M2E Not assigned,M3E Not assigned,M4E East Toronto (The Beaches),M5E Downtown Toronto (Berczy Park),M6E York (Caledonia-Fairbanks),M7E Not assigned,M8E Not assigned,M9E Not assigned
4,M1G Scarborough (Woburn),M2G Not assigned,M3G Not assigned,M4G East York (Leaside),M5G Downtown Toronto (Central Bay Street),M6G Downtown Toronto (Christie),M7G Not assigned,M8G Not assigned,M9G Not assigned


In [6]:
#Ponemos todo en una columna
canada = pd.melt(tabla)

#eliminamos la columna "variable"
canada.drop('variable', axis=1, inplace = True)

#lo dividimoa en las columnas que necesitamos
canada['PostalCode'] = canada['value'].str[:3]
canada['rest'] = canada['value'].str[3:]
canada['Borough'] = canada['rest'].str.split('(').str[0]
canada['Neighborhood'] = canada['rest'].str.split('(').str[1]

#que queden solo las 3 columnas que queremos
canada.drop('value', axis=1, inplace = True)
canada.drop('rest', axis=1, inplace = True)

#limpiamos, eliminamos los corchetes y dejamos , en vez de /
canada['Neighborhood'] = canada['Neighborhood'].str.replace(')', '')
canada['Neighborhood'] = canada['Neighborhood'].str.replace('/', ',')

#eliminamos las filas que sean "not assigned"
canada.drop(canada[canada['Borough'] == ' Not assigned'].index, inplace=True)
canada

<ipython-input-6-c3360ef19573>:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  canada['Neighborhood'] = canada['Neighborhood'].str.replace(')', '')


,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern , Rouge"
2,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
3,M1E,Scarborough,"Guildwood , Morningside , West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
...,...,...,...
170,M9N,York,Weston
171,M9P,Etobicoke,Westmount
172,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
175,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [7]:
canada.shape

(103, 3)

## Segunda parte

In [8]:
!pip install geocoder

In [ ]:
import geocoder

lat_lng_coords = None
canada["latitude"]= ""
canada["longitude"]= ""

for i in range(canada.shape[0]):
  g = geocoder.google('{},{}'.format(canada['PostalCode'][i],canada['Borough'][i] ))
  print(canada['PostalCode'][i],canada['Borough'] )
  lat_lng_coords = g.latlng

  canada['latitude'][i] = lat_lng_coords[0]
  canada['longitude'][i] = lat_lng_coords[1]